<a href="https://colab.research.google.com/github/zamoravm1/ict-for-smart-mobility/blob/main/Lab1_SmartMobility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratory 1 - Smart Mobility

## Preliminary data analysis

To get used to both MongoDB and the data at disposal, investigate first the collections and get used to the document and field stored in each.

#### Import section

In [ ]:
!pip install geopandas
!pip install mapclassify
import pymongo
import time
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from mapclassify import classify
# from google.colab import drive
# drive.mount('drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Create a mongo Client and a db

In [ ]:
client = pymongo.MongoClient('bigdatadb.polito.it:27017',
                     username='ictts',
                     password='Ict4SM22!',
                     authSource='carsharing',
                     authMechanism='SCRAM-SHA-1',
                     ssl=True,
                     tlsAllowInvalidCertificates=True)
db = client['carsharing']

#### Checking db format


In [ ]:
db["ActiveBookings"].find_one()

{'_id': ObjectId('5a2e8c0d2ad85324ef307f84'),
 'init_fuel': 39,
 'city': 'Austin',
 'walking': {'duration': -1, 'distance': -1},
 'vendor': 'car2go',
 'driving': {'duration': -1, 'distance': -1},
 'final_time': -1,
 'plate': 'FHB 2898',
 'engineType': 'CE',
 'init_time': 1512999938,
 'vin': 'WMEEJ3BA3FK799989',
 'smartPhoneRequired': False,
 'interior': 'GOOD',
 'final_fuel': -1,
 'exterior': 'GOOD',
 'init_date': datetime.datetime(2017, 12, 11, 7, 45, 38),
 'final_date': -1,
 'init_address': '4707 Harmon Ave, Austin, TX 78751, USA',
 'final_address': '',
 'origin_destination': {'type': 'LineString',
  'coordinates': [[-97.71452, 30.30632], [-1, -1]]},
 'public_transport': {'duration': -1, 'distance': -1, 'arrival_time': -1}}

### 1.1 How many documents are present in each collection?

In [ ]:
# Car2Go
ActiveBooking = db["ActiveBookings"]
ActiveParkings = db["ActiveParkings"]
PermanentBookings = db["PermanentBookings"]
PermanentParkings = db["PermanentParkings"]
# Enjoy
enjoy_ActiveBookings = db["enjoy_ActiveBookings"]
enjoy_ActiveParkings = db["enjoy_ActiveParkings"]
enjoy_PermanentBookings = db["enjoy_PermanentBookings"]
enjoy_PermanentParkings = db["enjoy_PermanentParkings"]

In [ ]:
print('Number of documents in ActiveBooking: ', ActiveBooking.count_documents({}))
print('Number of documents in ActiveParkings: ', ActiveParkings.count_documents({}))
print('Number of documents in PermanentBookings: ', PermanentBookings.count_documents({}))
print('Number of documents in PermanentParkings: ', PermanentParkings.count_documents({}))
print('Number of documents in enjoy_ActiveBookings: ', enjoy_ActiveBookings.count_documents({}))
print('Number of documents in enjoy_ActiveParkings: ', enjoy_ActiveParkings.count_documents({}))
print('Number of documents in enjoy_PermanentBookings: ', enjoy_PermanentBookings.count_documents({}))
print('Number of documents in enjoy_PermanentParkings: ', enjoy_PermanentParkings.count_documents({}))

Number of documents in ActiveBooking:  8743
Number of documents in ActiveParkings:  4790
Number of documents in PermanentBookings:  28180508
Number of documents in PermanentParkings:  28312676
Number of documents in enjoy_ActiveBookings:  0
Number of documents in enjoy_ActiveParkings:  0
Number of documents in enjoy_PermanentBookings:  6653472
Number of documents in enjoy_PermanentParkings:  6689979


### 1.2 Why the number of documents in PermanentParkings and PermanentBooking is similar?

Each car should had been booked and parking in some moment, indeed, in a perfect scenary, they should be the same, since PermanentParkings and PemanentBookings corresponds to longitudinal dataset with durations of car booking periods and duration of car parking periods respectly. 

### 1.3 For which cities the system is collecting data?

In [ ]:
cities = []
cities = list(set(ActiveBooking.distinct("city") + ActiveParkings.distinct("city") + PermanentBookings.distinct("city") + \
                  PermanentParkings.distinct("city") +  enjoy_ActiveBookings.distinct("city") + enjoy_ActiveParkings.distinct("city") + \
                  enjoy_PermanentBookings.distinct("city") + enjoy_PermanentParkings.distinct("city")))


cities=np.array(cities)
print("Including all collections:\n" + str(cities))

Including all collections:
['Rheinland' 'Seattle' 'Amsterdam' 'Bologna' 'Catania' 'Madrid'
 'Vancouver' 'Montreal' 'Twin Cities' 'Austin' 'Berlin' 'Portland' 'Roma'
 'Milano' 'Toronto' 'Denver' 'Hamburg' 'Firenze' 'Washington DC' 'Wien'
 'Munchen' 'New York City' 'Stuttgart' 'Calgary' 'Torino' 'Frankfurt'
 'Columbus' 'San Diego']


### 1.4 When the collection started and ended? 

In [ ]:
First_value_cursor =  PermanentBookings.find({}, {"init_date": 1}).sort("init_date", 1).limit(1)
import datetime

for x in First_value_cursor:
  print("First date: ", x['init_date'])

Last_value_cursor =  enjoy_PermanentBookings.find({}, {"init_date": 1}).sort("init_date", -1).limit(1)

for x in Last_value_cursor:
  print("Last date: ", x['init_date'])


### 1.5 What about the timezone of the timestamps?

Timestamp - GMT+1. 2016-12-13 18:40:42

Date - local zone. 2016-12-13 17:40:42



```
'init_date': datetime.datetime(2018, 1, 10, 15, 10, 35)
'init_time': 1515593435
```

In [ ]:
Torino_Bookings = PermanentBookings.find({'city' : 'Torino'})
for x in Torino_Bookings:
  print(x['init_date'],datetime.fromtimestamp(x['init_time']))
  break
  

## For each city:

### 1.6.1 What is the total number of cars seen in the whole period in each city? How does this relate to the fleet size at a given time?

In [ ]:
Torino_cars = PermanentBookings.distinct("plate", {'city' : 'Torino'})
print('Number of cars in Car2Go - Torino: ', len(Torino_cars))
enjoy_Torino_cars = enjoy_PermanentBookings.distinct("plate", {'city' : 'Torino'})
print('Number of cars in Enjoy - Torino: ' ,len(enjoy_Torino_cars))
Denver_cars = PermanentBookings.distinct("plate", {'city' : 'Denver'})
print('Number of cars in Car2Go - Denver: ', len(Denver_cars))

In [ ]:
start_date1 = datetime.fromisoformat('2017-01-01')
start_unixtime1 = (time.mktime(start_date1.timetuple()))

final_date1 = datetime.fromisoformat('2017-12-31')
final_unixtime1 = (time.mktime(final_date1.timetuple()))

car2go_Torino_cars2018 = PermanentBookings.distinct("plate", {'city' : 'Torino', 'init_time' : {"$gte": start_unixtime1, "$lte" : final_unixtime1}})
enjoy_Torino_cars2018 = enjoy_PermanentBookings.distinct("plate", {'city' : 'Torino', 'init_time' : {"$gte": start_unixtime1, "$lte" : final_unixtime1}})
Denver_cars2018 = PermanentBookings.distinct("plate", {'city' : 'Denver', 'init_time' : {"$gte": start_unixtime1, "$lte" : final_unixtime1}})
print('Number of cars in Car2Go - Torino: ' ,len(car2go_Torino_cars2018))
print('Number of cars in Enjoy - Torino: ' ,len(enjoy_Torino_cars2018))
print('Number of cars in Car2Go - Denver: ' ,len(Denver_cars2018))

2017

Number of cars in Car2Go - Torino:  609

Number of cars in Enjoy - Torino:  1552

Number of cars in Car2Go - Denver:  608

2018

Number of cars in Enjoy - Torino:  413

Number of cars in Enjoy - Torino:  641

Number of cars in Enjoy - Torino:  331

2019

Number of cars in Car2Go - Torino:  0

Number of cars in Enjoy - Torino:  328

Number of cars in Car2Go - Denver:  0


### 1.6.2 How many bookings have been recorded on the January 2018 in each city?

In [ ]:
start_date = datetime.fromisoformat('2018-01-01')
start_unixtime = (time.mktime(start_date.timetuple()))
final_date = datetime.fromisoformat('2018-01-31')
final_unixtime = (time.mktime(final_date.timetuple()))
Torino_Bookings = PermanentBookings.find({'city' : 'Torino', 'init_time' : {"$gte": start_unixtime, "$lte" : final_unixtime}})
n = 0 
for x in Torino_Bookings:
  n+= 1
print("Torino Bookings Jan 2018: ", n)

enjoy_Torino_Bookings = enjoy_PermanentBookings.find({'city' : 'Torino', 'init_time' : {"$gte": start_unixtime, "$lte" : final_unixtime}})
n = 0 
for x in enjoy_Torino_Bookings:
  n+= 1
print("enjoy Torino Bookings Jan 2018: ", n)

Denver_Bookings = PermanentBookings.find({'city' : 'Denver', 'init_time' : {"$gte": start_unixtime, "$lte" : final_unixtime}})
n = 0 
for x in Denver_Bookings:
  n+= 1
print("Denver Bookings Jan 2018: ", n)


# 2. Analysis of the data

Consider each city of your group, and the period of time of December 1st 2017 – February 28th 2018. Consider the time series (city, timestamp, duration, locations). Process it to further analyse it by producing the following plots and results:

In [ ]:
#Time period
start_date = datetime.fromisoformat('2017-12-01')
start_unixtime = (time.mktime(start_date.timetuple()))
final_date = datetime.fromisoformat('2018-02-28')
final_unixtime = (time.mktime(final_date.timetuple()))

dfs = []

# BOOKINGS DF
Torino_Bookings = PermanentBookings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'origin': {'$arrayElemAt' : ['$origin_destination.coordinates', 0]}, 'destination': {'$arrayElemAt' : ['$origin_destination.coordinates', 1]}}}])
df_bookingsTorino = pd.DataFrame(Torino_Bookings)
dfs.append(('Bookings Torino', df_bookingsTorino))

enjoy_Torino_Bookings = enjoy_PermanentBookings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'origin': {'$arrayElemAt' : ['$origin_destination.coordinates', 0]}, 'destination': {'$arrayElemAt' : ['$origin_destination.coordinates', 1]}}}])
df_bookingsEnjoyTorino = pd.DataFrame(enjoy_Torino_Bookings)
dfs.append(('Bookings Torino Enjoy', df_bookingsEnjoyTorino))

Denver_Bookings = PermanentBookings.aggregate([{'$match': {'city': 'Denver', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'origin': {'$arrayElemAt' : ['$origin_destination.coordinates', 0]}, 'destination': {'$arrayElemAt' : ['$origin_destination.coordinates', 1]}}}])
df_bookingsDenver = pd.DataFrame(Denver_Bookings)
dfs.append(('Bookings Denver', df_bookingsDenver))


# PARKINGS DF
Torino_Parkings = PermanentParkings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'location': '$loc.coordinates'}}])
df_parkingsTorino = pd.DataFrame(Torino_Parkings)
dfs.append(('Parkings Torino', df_parkingsTorino))

enjoy_Torino_Parkings = enjoy_PermanentParkings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'location': '$loc.coordinates'}}])
df_parkingsEnjoyTorino = pd.DataFrame(enjoy_Torino_Parkings)
dfs.append(('Parkings Torino Enjoy', df_parkingsEnjoyTorino))

Denver_Parkings = PermanentParkings.aggregate([{'$match': {'city': 'Denver', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'location': '$loc.coordinates'}}])
df_parkingsDenver = pd.DataFrame(Denver_Parkings)
dfs.append(('Parkings Denver', df_parkingsDenver))


#### 2.1 Derive the Cumulative Distribution Function of booking/parking duration and plot them. Which  consideration can you derive from the results? 

a. Which of the CDF is longer? Are there some outliers?
Booking. Yes. There is outliers due to short bookings.


b. Does the CDF change per each city? Why? Yes. the behaviors are differents.


c. Does the CDF change over time (e.g., aggregate per each week of data, or per each day or the week)? Why? yes.

In [ ]:
#Calculate and plot of the cdf
def cdf_plt(label, data):
  # getting data of the histogram
  tmp1 = np.cumsum(data)
  tmp2 = np.sum(data)
  cdf = tmp1/tmp2
  datas = np.sort(data)
  
  # plotting PDF and CDF
  plt.ylabel('CDF')
  plt.xlabel('Duration [min]')
  plt.grid(which='both')
  plt.xscale('log')
  plt.plot(datas, cdf, label=label)
  plt.legend()

In [ ]:
# plot of CDF in parking and booking for each city
for plot in range(3):
  plt.figure(plot)
  b = dfs[plot]
  p = dfs[3 + plot]
  cdf_plt(b[0], b[1]['duration'].values)
  cdf_plt(p[0], p[1]['duration'].values)
  plt.close

In [ ]:
#Separate each dataframe in weeks
start_date = datetime.fromisoformat('2017-12-01')
start_unixtime = (time.mktime(start_date.timetuple()))
final_date = datetime.fromisoformat('2018-02-28')
final_unixtime = (time.mktime(final_date.timetuple()))
week_unixtime = (time.mktime(datetime.fromisoformat('2022-10-23').timetuple())) - (time.mktime(datetime.fromisoformat('2022-10-16').timetuple()))
for tup in dfs:
  df = tup[1]
  conditions = []
  values = []
  for week in range(13):
    conditions.append((df['timestamp'] >= start_unixtime + week_unixtime*week) & (df['timestamp'] < start_unixtime + week_unixtime*(week+1)))
    values.append(week)
  df['Week'] = np.select(conditions, values)


In [ ]:
# Create a cdf for Weeks
for week in range(13):
  plt.figure(week)
  plt.title('Week' + str(week + 1))
  for tup in dfs:
    df = tup[1]
    cdf_plt(tup[0], df['duration'].loc[df['Week'] == week])
  plt.close

In [ ]:
def bar_per(labels, bookings, parkings, title, date):
  x = np.arange(len(labels)) + 1  # the label locations
  width = 0.35  # the width of the bars
  fig, ax = plt.subplots()
  rects1 = ax.bar(x - width/2, bookings, width, label='bookings')
  rects2 = ax.bar(x + width/2, parkings, width, label='parkings')

  # Add some text for labels, title and custom x-axis tick labels, etc.
  ax.set_title('Number of booked/parked cars per hour in ' + title + ' '+ date)
  ax.set_xlabel('Hour of the day')
  ax.set_ylabel('Number of booked/parked cars')
  ax.set_xticks(x)
  ax.set_xticklabels(labels, rotation=90)
  ax.legend()

  fig.tight_layout()
  plt.show()

### 2.2 Consider the system utilization over time: aggregate rentals per hour of the day, and then plot the  number of booked/parked cars (or percentage of booked/parked cars) per hour versus time of day. Do you notice any outliers? Can you explain them.

In [ ]:
#Separete each dataframe in hours of the day
date = '2017-12-01'
start_date = datetime.fromisoformat(date)
start_unixtime = (time.mktime(start_date.timetuple()))
for plot in range(3):
  hours = []
  occurencesb = []
  occurencesp = []
  b = dfs[plot]
  p = dfs[3 + plot]
  for hour in range(24):
    occurencesb.append(len(b[1].loc[(b[1]['timestamp'] > start_unixtime + hour*3600) & (b[1]['timestamp'] <= start_unixtime + (hour + 1)*3600)].values))
    occurencesp.append(len(p[1].loc[(p[1]['timestamp'] > start_unixtime + hour*3600) & (p[1]['timestamp'] <= start_unixtime + (hour + 1)*3600)].values))
    hours.append('0' + str(hour + 1) + ':00' if hour < 9 else str(hour + 1)  + ':00')
  title = dfs[plot][1]['city'].iloc[0] if plot != 1 else 'Enjoy ' + dfs[plot][1]['city'].iloc[0]
  bar_per(hours, occurencesb, occurencesp, title, date)
  plt.close

### 2.3 Derive a criterion to filter possible outliers (booking periods that are too short/too long), so to obtain  rentals from bookings, filtering system issues or problems with the data collection.

Data filtered by:
- Booking duration, min 3 minutes. Consequence of a user testing, system error
- Booking duration, max 180 minutes. Consequence of system error.  
- False rental, when the coordinates of initial and final position correspond. 
- Too short parking duration, consequence of a system error.

In [ ]:
# Duration filters

# Booking time
min_duration=5
max_duration=180
# False rental
#{'$match': {'moved':  True, 'duration':{'$gte': min_duration,'$lte': max_duration}}}

# BOOKINGS DF FILTERED

Torino_Bookings_filter=PermanentBookings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'moved': {'$ne': [{'$arrayElemAt' : ['$origin_destination.coordinates', 0]},{'$arrayElemAt' : ['$origin_destination.coordinates', 1]}]}}},{'$match': {'moved':  True, 'duration':{'$gte': min_duration,'$lte': max_duration}}},{'$project':{'city': 1,'duration':1,'cost':{'$multiply':['$duration',0.25]}}}])
df_bookingsTorino_filter = pd.DataFrame(Torino_Bookings_filter)
dfs.append(('Bookings Torino filtered',df_bookingsTorino_filter))

enjoy_Torino_Bookings_filter = enjoy_PermanentBookings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] },'moved': {'$ne': [{'$arrayElemAt' : ['$origin_destination.coordinates', 0]},{'$arrayElemAt' : ['$origin_destination.coordinates', 1]}]}}},{'$match': {'moved':  True, 'duration':{'$gte': min_duration,'$lte': max_duration}}}])
df_bookingsEnjoyTorino_filter = pd.DataFrame(enjoy_Torino_Bookings_filter)
dfs.append(('Bookings Torino Enjoy filtered', df_bookingsEnjoyTorino_filter))

Denver_Bookings_filter = PermanentBookings.aggregate([{'$match': {'city': 'Denver', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] },'moved': {'$ne': [{'$arrayElemAt' : ['$origin_destination.coordinates', 0]},{'$arrayElemAt' : ['$origin_destination.coordinates', 1]}]}}},{'$match': {'moved':  True, 'duration':{'$gte': min_duration,'$lte': max_duration}}}])
df_bookingsDenver_filter = pd.DataFrame(Denver_Bookings_filter)
dfs.append(('Bookings Denver filtered', df_bookingsDenver_filter))


In [ ]:
# Rentals per city

print('Total rentals Torino Car2go: '+ str(df_bookingsTorino_filter.index.stop))
print('Total rentals Torino Enjoy: '+ str(df_bookingsEnjoyTorino_filter.index.stop))
print('Total rentals Denver: '+ str(df_bookingsDenver_filter.index.stop))

### 2.4 Filtering data as above, consider the system utilization over time and the CDF of booking and parking  duration. How do these change? Are you able to filter outliers efficiently for both type of events? Consider also to plot the CDF of the filtered events. How do these appear?

In [ ]:
# plot of CDF in parking and booking for each city
for plot in range(3):
  plt.figure(plot)
  b = dfs[plot + 6]
  p = dfs[plot + 3]
  cdf_plt(b[0], b[1]['duration'].values)
  cdf_plt(p[0], p[1]['duration'].values)
  plt.close

### 2.5 Filtering the data as above, compute the average, median, standard deviation, and percentiles of the booking/parking duration over time (e.g., per each day of the collection).

In [ ]:
def metrics_plot(labels, mean, std, median, percentile):
  # getting data of the histogram
  plt.grid(which='both')
  plt.plot(labels, mean, label='Mean')
  plt.plot(labels, std, label='Std')
  plt.plot(labels, median, label='Median')
  plt.plot(labels, percentile, label='Percentile 90')
  plt.xticks(rotation='vertical')
  plt.legend()

In [ ]:
#Separete each dataframe in hours of the day
start_date = datetime.fromisoformat('2017-12-01')
start_unixtime = (time.mktime(start_date.timetuple()))
for plot in range(6):
  df = dfs[plot][1]
  mean = []
  std = []
  median = []
  percentile = []
  dates = []
  plt.figure()
  plt.title(dfs[plot][0] + ' Metrics')
  # 89
  for day in range(35):
    day_values = df['duration'].loc[(df['timestamp'] > start_unixtime + day*86400) & (df['timestamp'] <= start_unixtime + (day + 1)*86400)].values
    if len(day_values) > 0:
      mean.append(np.mean(day_values))
      std.append(np.std(day_values, ddof=1))
      median.append(np.median(day_values))
      percentile.append(np.percentile(day_values, q=90))
      date = datetime.fromtimestamp(start_unixtime + (day)*86400).strftime("%Y/%m/%d")
      dates.append(date)
  metrics_plot(dates,mean,std,median,percentile)
  plt.close

### 2.6 Consider one city of your collection and check the position of the cars when returned and compute the density of cars at rental ending time (the destination matrix) during different hours of the day.
### a. Plot the parking position of cars in different times using a mapping service of your preference. For instance, how different are the destination zones on Mondays between 8-10 and 18-20? Or same time, but different day? Or weekend and weekdays?

In [ ]:
start_date = datetime.fromisoformat('2017-12-04 00:00:00')
start_unixtime = (time.mktime(start_date.timetuple()))
print(start_unixtime)
print(start_date)

In [ ]:
# Export location data
start_date = datetime.fromisoformat('2017-12-04')
start_unixtime = (time.mktime(start_date.timetuple()))
print(start_unixtime)
final_date1 = datetime.fromisoformat('2017-12-04')
final_unixtime1 = (time.mktime(final_date1.timetuple()))

# PermanentParkings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'location': '$loc.coordinates'}}])
Torino_Locations = PermanentParkings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': 1512374400, '$lte': 1512381600 } }},{'$project': {'location': '$loc.coordinates'}}])
df_locationsTorino = pd.DataFrame(Torino_Locations)
dfs.append(('Destinations Torino', df_locationsTorino))
print(df_locationsTorino)

In [ ]:
start_unixtime = 1512410400
final_unixtime = 1512417600

Torino_Locations = PermanentParkings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime }}},
                                                {'$project': {
                                                              'lng': {'$arrayElemAt' : ['$loc.coordinates',0]},
                                                              'lat': {'$arrayElemAt' : ['$loc.coordinates',1]}
                                                             }}])
df_locationsTorino = pd.DataFrame(Torino_Locations)
dfs.append(('Destinations Torino', df_locationsTorino))
df_locationsTorino['_id'] = df_locationsTorino['_id'].map(lambda x : x.__str__())

In [ ]:
geometry = gpd.points_from_xy(df_locationsTorino["lng"],df_locationsTorino["lat"],crs="EPSG:4326")
gdf = GeoDataFrame(df_locationsTorino, geometry=geometry)   
gdf.explore()